In [1]:
# import os
# os.environ["GOOGLE_API_KEY"] = ""
# os.environ["ANTHROPIC_API_KEY"] = ""
# os.environ["LANGCHAIN_TRACING_V2"] = ""
# os.environ["LANGCHAIN_API_KEY"]=""

from dotenv import load_dotenv
load_dotenv()

True

## Prompts

In [2]:
environmental_template = """You are a passionate environmentalist with a deep understanding of ecological systems and sustainability principles. \
You excel at providing insightful explanations and solutions related to environmental issues in a clear and engaging manner. \
When faced with a question outside your expertise, you gracefully acknowledge your limitations.\

Answer the following question based on the provided context:

<context>
{context}
</context>

Here is the question:
{input}"""

education_template = """You are an enthusiastic and knowledgeable educator with a profound understanding of pedagogy and learning principles. \
You are adept at delivering comprehensive explanations and fostering engaging learning experiences. 

Answer the following question with the provided context.

<context>
{context}
</context>

Here is the question:
{input}"""

In [3]:
prompt_infos = [
{
    "name": "environmentalist",
    "description": "Ideal for addressing environmental concerns and offering insightful solutions",
    "prompt_template": environmental_template,
},
{
    "name": "education", 
    "description": "Good for answering questions related to education or study or 早自習", 
    "prompt_template": education_template,
}]

In [4]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
你是一個router，你的工作是分析input的問題與甚麼領域有關，\
當問題不清楚時適當修改問題，接著把問題傳遞給適當的language model.\
你會得到所有可以選擇的language model名稱及它們擅長哪個領域的問題。\

You can also use the history messages provided here to make your decision.
<< HISTORY >>
{{history}}

<< INPUT >>
{{input}}


<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:

\```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ summary of the history relevant to the topic of input
}}}}
\```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is completely\
unrelated to any of the candidate prompts.
REMEMBER: "next_inputs" should contain a concise summary of the history to provide context for the input.


<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

## LLM

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

gemini = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro", convert_system_message_to_human=True)
claude = ChatAnthropic(temperature=0,  model_name="claude-3-haiku-20240307")
# openai = ChatOpenAI()

c:\Users\ethan\miniconda3\envs\genai-course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document loader & text splitter

In [6]:
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector = Chroma.from_documents(documents, embedding_function)

In [ ]:
vector.delete_collection()

## Destinations

In [7]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [8]:
prompt = ChatPromptTemplate.from_template(template=environmental_template)

document_chain = create_stuff_documents_chain(claude, prompt)
retriever = vector.as_retriever(search_kwargs={"k": 1})

enviromental_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
prompt = ChatPromptTemplate.from_template(template=education_template)

document_chain = create_stuff_documents_chain(claude, prompt)
retriever = vector.as_retriever(search_kwargs={"k": 1})

education_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

general_chain = RunnablePassthrough.assign(
    answer=(ChatPromptTemplate.from_template("{history} {input}") | claude | StrOutputParser())
)

## Route to Destination

In [11]:
def route(info):
    print(info)
    if info["destination"]:
        if "environmentalist" in info["destination"].lower():
            return enviromental_chain
        elif "education" in info["destination"].lower():
            return education_chain
    else:
        return general_chain

In [12]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# destinations
# ['physics: Good for answering questions about physics',
#  'math: Good for answering math questions']

router_system_template = PromptTemplate.from_template(MULTI_PROMPT_ROUTER_TEMPLATE)
router_system_template = router_system_template.format(destinations=destinations_str)

router_system_template = HumanMessagePromptTemplate.from_template(router_system_template)

## Memory 

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(llm=gemini, max_token_limit=1024, ai_prefix="")
memory.load_memory_variables({})

{'history': ''}

In [14]:
router_prompt = ChatPromptTemplate.from_messages(
    [
        # MessagesPlaceholder(variable_name="history", optional=True), # Put history into router llm ???
        router_system_template,
    ]
)

## Build Chain

In [15]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.router.llm_router import RouterOutputParser
from operator import itemgetter

router_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | router_prompt
    | claude
    | RouterOutputParser()
    | {"destination": itemgetter("destination"), "next_inputs": lambda x: x["next_inputs"]['input']} 
)

In [16]:

multi_prompt_chain = (
    RunnablePassthrough.assign(
        route=router_chain
    )
    | { 
        "destination": lambda x: x["route"]["destination"], 
        "input": lambda x: f'{x["route"]["next_inputs"]}\n{x["input"]}'
    } 
    | RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    ) 
    | RunnableLambda(route)
) 

## Usage

In [17]:
from api.server import Server
from api.manager import Manager
from api.models.match import Match

server = Server("http://localhost:3000", "f2a87978-7574-4cd6-ae3f-a940d3386315")
manager = Manager(server)

In [18]:
print("all matches:")
matches = manager.get_match_list()
for match in matches:
    print(match)
print("-" * 80)
manager.set_current_match(2)

all matches:
Match ID: 1, Name: testgame, Players: [], History Messages: []
Match ID: 2, Name: testgame, Players: [], History Messages: []
--------------------------------------------------------------------------------


In [19]:
print("current history:")
match = manager.get_current_match()
for msg in match.history_msgs:
    print(msg)

current history:


In [20]:
memory.clear()

In [21]:
inputs = []

inputs.append({"input": 
"""兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?
正方：支持廢除早自習
反方：反對廢除早自習

作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。
正方:"""})

inputs.append({"input": 
"""作為反方，請堅守反對廢除早自習的立場，簡明扼要地陳述你的意見和理由。
反方:"""
})

inputs.append({"input": 
"""作為正方，請堅守支持廢除早自習的立場。請針對反對方所發表的意見，進行反駁。
正方:"""})

inputs.append({"input": 
"""作為反方，請堅守反對廢除早自習的立場。請針對正方所發表的意見，進行反駁。
反方:"""})

In [28]:
import time

local_match_progress = 0
first = False

while True:

    match = manager.get_current_match()

    # init 
    if local_match_progress == 0 and first:
        # respond
        current_input = inputs[local_match_progress]
        response = multi_prompt_chain.invoke(current_input)
        answer = response.get("answer")
        
        # save 
        print(answer)
        memory.save_context(current_input, {"output": answer})
        local_match_progress += 1
        
        if (local_match_progress == 4):
            print("End")
            break

        # push
        manager.send_message(answer)

    # lag behind remote history: sync with server
    if local_match_progress < len(match.history_msgs):
        assert local_match_progress == len(match.history_msgs) - 1

        # pull
        new_messages = match.history_msgs[local_match_progress]
        print(new_messages.text)

        # save
        current_input = inputs[local_match_progress]
        memory.save_context(current_input, {"output": new_messages.text})
        local_match_progress += 1

        if (local_match_progress == 4):
            print("End")
            break
        
        # respond
        current_input = inputs[local_match_progress]
        response = multi_prompt_chain.invoke(current_input)
        answer = response.get("answer")

        # save 
        print(answer)
        memory.save_context(current_input, {"output": answer})
        local_match_progress += 1

        # push
        manager.send_message(answer)

        if (local_match_progress == 4):
            print("End")
            break

    time.sleep(5)
    
    

作為正方,我們堅持支持廢除早自習的立場,並提出以下幾點理由:

1. 早自習對學生的身心健康造成負擔。長時間的自習會增加學生的壓力和焦慮,影響他們的睡眠和休息,從而影響身心發展。我們應該給學生更多的自由時間,讓他們有機會參與其他有益的活動,如運動、興趣小組等,以促進全面發展。

2. 早自習並非最有效的學習方式。學生在早自習時往往缺乏老師的指導和互動,很難保證學習效果。相反,上課時老師的講解和課堂討論更有助於知識的吸收和理解。我們應該把有限的時間用在更有價值的教學活動上。

3. 早自習會加重學生的學習負擔。學生在上課、完成作業、參加課外活動之外,還要花大量時間在早自習上,這對他們來說是一種過度的要求。我們應該減輕學生的學習壓力,讓他們有更多時間休息和娛樂。

4. 早自習並非所有學生都需要。不同學生的學習能力和需求各不相同,強制性的早自習並不能照顧到每個學生的個別差異。我們應該根據學生的實際情況,採取更靈活的教學方式。

綜上所述,我們認為廢除早自習是一個明智的決定,能夠更好地促進學生的全面發展。我們呼籲大家支持這一改革,共同為學生創造更好的學習環境。
{'destination': 'education', 'input': "The discussion is about whether to abolish the early self-study (早自習) system in schools. The proposing side argues that early self-study negatively impacts students' physical and mental health, while the opposing side believes it helps cultivate students' self-learning abilities and good study habits.\n作為反方，請堅守反對廢除早自習的立場，簡明扼要地陳述你的意見和理由。\n反方:", 'history': 'Human: 兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?\n正方：支持廢除早自習\n反方：反對廢除早自習\n\n作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。\n正方:\n: 作為正方,我們堅

In [ ]:
inputs = {"input": 
"""兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?
正方：支持廢除早自習
反方：反對廢除早自習

作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。
正方:"""}
response = multi_prompt_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": 
"""作為反方，請堅守反對廢除早自習的立場，簡明扼要地陳述你的意見和理由。
反方:"""
}
response = multi_prompt_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": 
"""作為正方，請堅守支持廢除早自習的立場。請針對反對方所發表的意見，進行反駁。
正方:"""}
response = multi_prompt_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": 
"""作為反方，請堅守反對廢除早自習的立場。請針對正方所發表的意見，進行反駁。
反方:"""}
response = multi_prompt_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

## Plot

In [ ]:
multi_prompt_chain.get_graph().print_ascii()